This code is designed to merge the credits dataframe and the tmdb details dataframe into a dataframe called merged_movie_details_credits.csv

In [1]:
!pip install tmdbv3api

In [2]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
from tmdbv3api.exceptions import TMDbException
import random
import pandas as pd
import csv
tmdb=TMDb()
tmdb.api_key='9cf68f4c97c8f0cc6bb9646da389a808'

>The next cell will fetch the actor's star score in the year of the movie's release by their id and apply the predefined weighting function to modify the star score based on the billing order.

In [3]:
def calculate_star_scores(billing_order,star_score):
    # Define the shape parameters
    X_0 = 4
    k = 1.5
    Z = k * (billing_order - X_0)
    # Calculate the star score using the logistic function (can be modified)
    mod = 1 - 1 / (1 + np.exp(-Z))
    star_score_mod=star_score*mod
    return star_score_mod

In [6]:
import pandas as pd
import csv
import numpy as np
from math import exp

# Initialize csv file paths
csv_file_path1='../../../3. Data Preparation/3.1 Select Data/people_info_star_scores_merged.csv' 
csv_file_path2='MISSING_merged_data_with_revenues_with_release_years.csv'
# Create DataFrames from both CSV files
df1 = pd.read_csv(csv_file_path1)
df2 = pd.read_csv(csv_file_path2)

In [7]:
from datetime import datetime
import pandas as pd
import ast
import numpy as np

star_scores = []  # Create a list to collect star scores

for index, row in df2.iterrows():
    row_cast_info = row['cast_ids']  # Extract cast_ids array
    row_cast_info = ast.literal_eval(row_cast_info)
    star_scores_row = []
    for cast_id in row_cast_info:
        release_date=row['Release Year']
        star_column_name = f'star_{release_date}' # Create the corresponding column name
        cast_info_row = df1[df1['id'] == cast_id] # Get the row containing the queried cast id
        star_score = cast_info_row[star_column_name].values[0] # Get the star score from that row corresponding to the correct column
        star_scores_row.append(star_score)
    star_scores_mod = []  # Initialize a list to store modified star scores
    for index, star_score in enumerate(star_scores_row):
        billing_order = index + 1  # Index starts from 0, the equation needs it to start from 1
        star_score_mod = calculate_star_scores(billing_order, star_score) # Apply the above function to the star score
        star_scores_mod.append(star_score_mod) # Add the change to a row array
    star_scores.append(star_scores_mod) # Append that row array to a singular entry in the corresponding row in df2
# Create an output dataframe
df_output = df2
df_output['star_scores'] = star_scores

# Save to csv
df_output.to_csv('MISSING_merged_data_mod_star_score.csv', index=False)